In [1]:
import os
import glob

In [2]:
def get_entity_dict(a1_file):
    entity_dict={}
    with open(a1_file, 'r') as reader:
        for line in reader:
            tab_del_list=line.strip().split("\t")
            entity_id = tab_del_list[0]
            entity_type = tab_del_list[1].split(' ')[0]
            start_off = int(tab_del_list[1].split(' ')[1])
            end_off = int(tab_del_list[1].split(' ')[2])
            entity_name = tab_del_list[2]
            
            #print entity_id, entity_type,start_off, end_off,entity_name
            if entity_id not in entity_dict:
                entity_dict[entity_id] = [entity_type, start_off, end_off,entity_name]
            elif entity_id in entity_dict:
                raise Exception('key error')
                
    return entity_dict

In [3]:
def get_event_dict(a2_file):
    event_dict={}
    with open(a2_file, 'r') as reader:
        for line in reader:
            if 'E' in line.strip().split('\t')[0]:
                event_id = line.strip().split('\t')[0]
                if event_id not in event_dict:
                    event_dict[event_id]=line.strip().split('\t')[1].split(' ')
                else:
                    raise Exception('key error')  

                
    return event_dict

In [4]:
def extract_entity_relation(line, event_dict, entity_dict):
    if 'E' in line.split('\t')[0]:
        #print line.split('\t')[0],line.split('\t')[1].split(' ')
        if len(line.split('\t')[1].split(' '))>=3:
            theme_string = line.split('\t')[1].split(' ')[1].split(':')[1]
            cause_string = line.split('\t')[1].split(' ')[2].split(':')[1]
            
            #print line
            if 'E' in theme_string:
                effect = event_dict[theme_string][0].split(':')[0]
                result_entity_id = event_dict[theme_string][1].split(':')[1]

                if 'E' in cause_string:
                    cause_entity_id = event_dict[cause_string][1].split(':')[1]
                    if 'T' in cause_entity_id and 'T' in result_entity_id:
                        return entity_dict[cause_entity_id][3] , effect ,  entity_dict[result_entity_id][3]
                
                elif 'E' not in cause_string :
                    cause_entity_id = line.split('\t')[1].split(' ')[2].split(':')[1]
                    if 'T' in result_entity_id:
                        return entity_dict[cause_entity_id][3] , effect , entity_dict[result_entity_id][3]

            elif 'E' in cause_string:
                    effect = line.split('\t')[1].split(' ')[0].split(':')[0]
                    result_entity_id = theme_string
                    cause_entity_id = event_dict[cause_string][1].split(':')[1]
                    
                    if 'T' in cause_entity_id: 
                        return entity_dict[cause_entity_id][3] , effect , entity_dict[result_entity_id][3]


def preprocess(a1_file, a2_file):
    entity_dict = get_entity_dict(a1_file)
    event_dict = get_event_dict(a2_file)
    
    result_list =[]
    
    with open(a2_file, 'r') as reader:
        for line in reader:
            line = line.strip()
            result = extract_entity_relation(line,event_dict,entity_dict)
            if result is not None:
                result_list.append(result)
                
    return result_list

In [6]:
text = './data/BioNLP-ST-2013_GE_train_data_rev3/PMC-1310901-01-INTRODUCTION.txt'
a1 = './data/BioNLP-ST-2013_GE_train_data_rev3/PMC-1310901-01-INTRODUCTION.a1'
a2 = './data/BioNLP-ST-2013_GE_train_data_rev3/PMC-1310901-01-INTRODUCTION.a2'

In [146]:
preprocess(a1, a2)

In [147]:
dataset_folder = './data/BioNLP-ST-2013_GE_train_data_rev3/'

In [171]:
file_list = list(set(map(lambda x:x.split('.')[0],os.listdir(dataset_folder))))
file_list.remove('LICENSE')
file_list.remove('README')
path_list = map(lambda x:dataset_folder+x , file_list)

In [226]:
total_list=[]
for path in path_list:
    a1_file = path+'.a1'
    a2_file = path+'.a2'
    total_list.extend(preprocess(a1_file, a2_file))

In [229]:
total_list

[('IL-2', 'Gene_expression', 'IL-2'),
 ('IL-2', 'Localization', 'IL-2'),
 ('Bcl10', 'Protein_catabolism', 'Bcl10'),
 ('PKCalpha', 'Phosphorylation', 'p65'),
 ('Foxp3', 'Binding', 'CREB-1'),
 ('Foxp3', 'Binding', 'activating transcription factor 2'),
 ('p300', 'Transcription', 'Gal4-BD-CREB-1'),
 ('Foxp3', 'Positive_regulation', 'Gal4-BD-CREB-1'),
 ('Foxp3', 'Positive_regulation', 'Gal4-BD-CREB-1'),
 ('p300', 'Binding', 'p300'),
 ('Foxp3', 'Binding', 'p300'),
 ('Foxp3', 'Binding', 'p300'),
 ('LMP1', 'Positive_regulation', 'Akt'),
 ('Akt', 'Negative_regulation', 'FoxO1'),
 ('STAT4', 'Gene_expression', 'IL-10'),
 ('STAT4', 'Gene_expression', 'IFN-gamma'),
 ('STAT4', 'Gene_expression', 'IL-4'),
 ('STAT4', 'Gene_expression', 'IL-10'),
 ('STAT4', 'Gene_expression', 'IL-10'),
 ('IL-12', 'Gene_expression', 'IL-10'),
 ('OVA', 'Gene_expression', 'IL-10'),
 ('STAT6', 'Gene_expression', 'IL-4'),
 ('STAT6', 'Gene_expression', 'IL-10'),
 ('STAT4', 'Gene_expression', 'IL-10'),
 ('IFN-gamma', 'Regulat

In [12]:
import nltk.data
import re
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [23]:
def get_sentence_list(text_file):
    sentence_list = []
    with open(text_file,'r') as reader:
        for i,line in enumerate(reader):
            temp_list = tokenizer.tokenize(line.strip())
            sentence_list.extend(temp_list)  
    return sentence_list

In [24]:
def get_entity_set(a1_file):
    with open(a1_file ,'r') as reader:
        entity_set = set([line.strip().split('\t')[2] for line in reader])
    return entity_set

In [25]:
def get_word_positions(string, word):
    position_list=[]
    for m in re.finditer(word, string):
        position_list.append((m.start(),m.end()))
    return position_list

In [31]:
def insert_entitiy_notation(text_file,a1_file):
    entity_dict = get_entity_dict(a1_file)
    entity_set = get_entity_set(a1_file)
    sentence_list = get_sentence_list(text_file)
    
    for sentence in sentence_list:
        for entity in entity_set:
            position_list = get_word_positions(sentence,entity)
            if len(position_list)>=1:
                print sentence, entity, position_list

In [32]:
%time insert_entitiy_notation(text, a1)

The hallmark genetic aberration of CML is a reciprocal chromosomal translocation t(9;22) leading to expression of a bcr-abl fusion gene, an aberrant activated tyrosine kinase (2). bcr-abl fusion gene [(116, 135)]
The hallmark genetic aberration of CML is a reciprocal chromosomal translocation t(9;22) leading to expression of a bcr-abl fusion gene, an aberrant activated tyrosine kinase (2). bcr [(116, 119)]
Treatment with interferon alpha (IFN-alpha) prolongs survival of CML patients and is associated with a complete cytogenetic response in 5-33% of CML patients (1,2). interferon alpha [(15, 31)]
Treatment with interferon alpha (IFN-alpha) prolongs survival of CML patients and is associated with a complete cytogenetic response in 5-33% of CML patients (1,2). IFN-alpha [(33, 42)]
Recently, we described an impaired expression of the interferon regulatory factor 4 (IRF-4) in CML, correlating with poor response to IFN-alpha treatment (3). IFN-alpha [(134, 143)]
Recently, we described an imp